# Advanced 3D language

GeoLIPI can be used to also define primitives as used in [GeoCODE](https://threedle.github.io/GeoCode/), [SIF](https://arxiv.org/abs/1904.06447), and other 2D-to-3D operators (extrusions/revolutions etc.). This notebook shows examples of such primitives in action.

In [ ]:
import sys
sys.path.append('../')

import numpy as np
import torch as th
import matplotlib.pyplot as plt

import geolipi.symbolic as gls
from geolipi.torch_compute.sphere_marcher import Renderer

dtype = th.float32
device = th.device("cuda")
resolution = (512, 256)


In [ ]:
# Just use the default settings in the renderer
renderer = Renderer(resolution=resolution, device=device, 
            dtype=dtype, unroll_expression=False, torch_compile=False)


In [ ]:
# Advanced Primitives: Extrusion based primitives.
expr_set = []
p1 = (0.1, 0.3)
p2 = (-0, 0, 0)
p3 = (-0, -2, 0.0)
for i in range(6):
    j = i + 1
    dist = 2#  * j
    theta = np.pi / 3  * j
    x = dist * np.cos(theta)
    y = dist * np.sin(theta)

    expression = gls.QuadraticBezierExtrude3D(
                    gls.Box2D(p1),
                    p2,
                    (0, 0.5, 0),
                    (x, -0.2, y),
                    (0,))
    expr_set.append(expression)

    theta = np.pi/6 + np.pi / 3  * j
    x = dist * np.cos(theta)
    y = dist * np.sin(theta)
    expression = gls.QuadraticBezierExtrude3D(
                    gls.Box2D(p1),
                    p2,
                    (0, -0.5, 0),
                    (-x, 0.2, -y),
                    (0,))
    expr_set.append(expression)

expression = gls.Union(*expr_set)

expression = expression.tensor().cuda()

image = renderer.render(expression)
plt.figure(figsize=(10, 5))
plt.imshow(image.detach().cpu().numpy())
plt.axis('off')


In [ ]:
# Advanced Primitive: Gaussian based
expr_set = []
for i in range(30):
    position = 0.5 + i * 0.1
    theta = np.pi / 5 * i
    size = (0.1, 0.1)
    expr = gls.InexactAnisotropicGaussian3D(
        (position * np.cos(theta), 0, position * np.sin(theta)),
        (size[0], 0.1, size[1]),
        (-1,))
    expr_set.append(expr)

expr = gls.Translate3D(gls.Union(*expr_set), (-1, 0, 1))
expr = gls.Erode3D(expr, 0.07)
# expr = gls.Sphere3D((0.1,))
expr = expr.tensor().cuda()
print(expr.pretty_print())
image = renderer.render(expr,)
plt.figure(figsize=(10, 5))
plt.imshow(image.detach().cpu().numpy())
plt.axis('off')


In [ ]:
# Advanced Primitive: Revolution & Extrusion of 2D SDFs.

expr = gls.Circle2D((0.2,))
# expr = gls.SimpleExtrusion3D(expr, (0.7))
expr = gls.Revolution3D(expr, (1.2))
expr = gls.EulerRotate3D(expr, (np.pi/2, 0, 0))
expr = expr.tensor().cuda()

print(expr.pretty_print())
image = renderer.render(expr,)
plt.figure(figsize=(10, 5))
plt.imshow(image.detach().cpu().numpy())
plt.axis('off')